In [6]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [7]:
AGENT_NAME = "ab010-multiturn"

In [8]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.

from typing import cast
import uuid

from agentex.types import Task

TASK_ID = str(uuid.uuid4())[:8]

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="task/create",
    params={
        "name": f"{TASK_ID}-task",
        "params": {}
    }
)

task = cast(Task, rpc_response.result)
print(task)

Task(id='e5b17323-4b7b-4043-874a-33a64aec8ab1', created_at=datetime.datetime(2025, 7, 27, 1, 16, 34, 635866, tzinfo=TzInfo(UTC)), name='36c97177-task', status='RUNNING', status_reason='Task created, forwarding to ACP server', updated_at=datetime.datetime(2025, 7, 27, 1, 16, 34, 635866, tzinfo=TzInfo(UTC)))


In [9]:
# Test non streaming response
from typing import cast
from agentex.types import Event

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.rpc_by_name(
    agent_name=AGENT_NAME,
    method="event/send",
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = cast(Event, rpc_response.result)
print(event)

Event(id='a1835392-e2d7-43a2-a476-aeca90084653', agent_id='079830a3-8402-4b82-a4f6-bac497f87a10', sequence_id=211, task_id='e5b17323-4b7b-4043-874a-33a64aec8ab1', content=TextContent(author='user', content='Hello what can you do?', attachments=None, format='plain', style='static', type='text'), created_at=datetime.datetime(2025, 7, 27, 1, 16, 35, 496828, tzinfo=TzInfo(UTC)))


In [10]:
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)

╭───────────────────────── USER [07/27/2025 01:16:35] ─────────────────────────╮
│ Hello what can you do?                                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────── AGENT [07/27/2025 01:16:38] ─────────────────────────╮
│ Hello! I can assist you with a variety of tasks, including:                  │
│                                                                              │
│  1 Answering Questions: I can provide information on a wide range of topics, │
│    including science, history, technology, and more.                         │
│  2 Providing Explanations: I can explain complex concepts in simpler terms.  │
│  3 Writing Assistance: I can help with writing essays, articles, or creative │
│    writing, including brainstorming ideas and editing.                       │
│  4 Language Help: I can assist with language learning, grammar questions,    │
│    and translations.                                                         │
│  5 Recommendations: I can offer suggestions for books, movies, or other      │
│    resources based on your interests.                                        │
│  6 Problem Solving: I can help you work through problems, whether they’re    │
│    academic or practical in nature.                                          │
│  7 General Advice: I can offer tips and guidance on various topics, such as  │
│    study techniques or productivity.                                         │
│                                                                              │
│ Feel free to ask me anything specific that you need help with!               │
╰──────────────────────────────────────────────────────────────────────────────╯

Streaming timed out after 5 seconds - returning collected messages
